In [1]:
import pandas as pd
from utils import tool, indexing
import numpy as np

/home/fizh/miniconda3/envs/rtd/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

## Load Dataset

In [ ]:
file_path = "static/data SOAP indo.xlsx"
rekmed_xl = pd.read_excel(file_path)

In [ ]:
rekmed_xl.isnull().sum()

## Preprocessing

In [ ]:
rekmed_xl = tool.drop_unnecessary_samples(rekmed_xl)
rekmed_xl = tool.remove_html_tag(rekmed_xl)

In [ ]:
rekmed_xl.isnull().sum()

In [ ]:
# rekmed_xl.reset_index(drop=False)
rekmed_xl

In [21]:
# save preprocessing data
# tool.save_py_obj("rekmed_post_preprocess.pkl", rekmed_xl)
# load previous clean data
rekmed_xl_pkl = tool.load_py_obj("rekmed_post_preprocess.pkl")

## Generate Question & Answer

In [64]:
# sampling
# 1000 samples
# sample_rekmed = rekmed_xl_pkl[:1261]
# 500 samples
# sample_rekmed = rekmed_xl_pkl[:633]
# 200 samples
# sample_rekmed = rekmed_xl_pkl[:245]
# 100 samples
sample_rekmed = rekmed_xl_pkl[:118]
print(f"total samples = {len(sample_rekmed)}")
rekmed_l_tokens = indexing.tokenization(sample_rekmed.diagnosa)
sample_rekmed, rekmed_l_tokens = tool.drop_meaningless_tokens(sample_rekmed, rekmed_l_tokens)
print(f"total samples after dropping some meaningless tokens = {len(sample_rekmed)}")

# all data
# print(len(rekmed_xl_pkl))
# rekmed_l_tokens = indexing.tokenization(rekmed_xl_pkl.diagnosa)
# sample_rekmed, rekmed_l_tokens = tool.drop_meaningless_tokens(rekmed_xl_pkl, rekmed_l_tokens)
# print(f"total samples after dropping some meaningless tokens = {len(sample_rekmed)}")

total samples = 118


100%|██████████| 118/118 [00:00<00:00, 55879.85it/s]

drop 18 samples
total samples after dropping some meaningless tokens = 100



/home/fizh/projects/rtd.rekmed/utils/tool.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(detect_index, inplace=True)


In [65]:
# # for all data
# QA = tool.qa_generator(rekmed_xl_pkl)
# samples data
QA = tool.qa_generator(sample_rekmed)
QA_test = tool.qa_generator_test(sample_rekmed)

100%|██████████| 100/100 [00:00<00:00, 1711.03it/s]


In [69]:
# tool.save_py_obj("QA_qa_generator.pkl", QA)
QA = tool.load_py_obj("QA_qa_generator.pkl")
# tool.save_py_obj("QA_qa_generator_test.pkl", QA_test)
QA_test = tool.load_py_obj("QA_qa_generator_test.pkl")

In [70]:
QA = tool.grouping_qa(QA)
QA_test = tool.grouping_qa(QA_test)

100%|██████████| 645/645 [00:00<00:00, 50483.80it/s]


drop 0 samples
drop 0 samples


100%|██████████| 129/129 [00:00<00:00, 63895.28it/s]


drop 0 samples
drop 0 samples


100%|██████████| 129/129 [00:00<00:00, 582416.81it/s]


In [3]:
# tool.save_py_obj("QA_grouping_qa.pkl", QA)
QA = tool.load_py_obj("QA_grouping_qa.pkl")
# tool.save_py_obj("QA_grouping_qa_test.pkl", QA_test)
QA_test = tool.load_py_obj("QA_grouping_qa_test.pkl")

In [73]:
label_entailment = tool.gather_entailment(QA) # 0.8

100%|██████████| 645/645 [00:19<00:00, 32.38it/s]


In [74]:
label_neutral = tool.gather_neutral(QA) # 0.4

100%|██████████| 645/645 [00:19<00:00, 33.01it/s]


In [75]:
label_contradiction = tool.gather_contradiction(QA) # 0

100%|██████████| 645/645 [00:34<00:00, 18.79it/s]


In [25]:
# tool.save_py_obj("label_entailment.pkl", label_entailment)
# tool.save_py_obj("label_neutral.pkl", label_neutral)
# tool.save_py_obj("label_contradiction.pkl", label_contradiction)

label_entailment = tool.load_py_obj("label_entailment.pkl")
label_neutral = tool.load_py_obj("label_neutral.pkl")
label_contradiction = tool.load_py_obj("label_contradiction.pkl")

In [26]:
print(f"Total samples of QA training : {len(QA)}")
print(f"Total samples of QA testing : {len(QA_test)}")
print(f"entailment : {len(label_entailment)}")
print(f"neutral : {len(label_neutral)}")
print(f"contradiction : {len(label_contradiction)}")

Total samples of QA training : 645
Total samples of QA testing : 129
entailment : 555
neutral : 1666
contradiction : 3200


In [80]:
train_examples = tool.concatenate_train_examples(
    label_entailment, label_neutral, label_contradiction
)

In [28]:
# tool.save_py_obj("train_examples.pkl", train_examples)
train_examples = tool.load_py_obj("train_examples.pkl")

## Training

In [83]:
model_name = "microsoft/mpnet-base"
sts_model = tool.STSModel(model_name, local=False)

2022-11-30 00:24:04 - Load pretrained SentenceTransformer: microsoft/mpnet-base
2022-11-30 00:24:11 - No sentence-transformers model found with name /home/fizh/.cache/torch/sentence_transformers/microsoft_mpnet-base. Creating a new one with MEAN pooling.


Some weights of the model checkpoint at /home/fizh/.cache/torch/sentence_transformers/microsoft_mpnet-base were not used when initializing MPNetModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetModel were not initialized from the model checkpoint at /home/fizh/.cache/torch/sentence_transformers/microsoft_mpnet-base and are newly initialized: ['mpnet.pooler.dense.weight', 'mpnet.pooler.dense.bias

2022-11-30 00:24:13 - Use pytorch device: cuda


In [84]:
# prepare training & validation samples
sts_model.get_train_test_dataset(train_examples)

In [85]:
sts_model.fit()

2022-11-30 00:24:57 - Read Train Dataset
2022-11-30 00:24:57 - Read Validation Dataset
2022-11-30 00:24:57 - Warmup-steps: 58


                                            
Epoch:   0%|          | 0/4 [00:10<?, ?it/s]               

2022-11-30 00:25:08 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 14 steps:


                                            
                                                           
                                                           
                                                           
                                                           
Epoch:   0%|          | 0/4 [00:12<?, ?it/s]               

2022-11-30 00:25:09 - Cosine-Similarity :	Pearson: -0.0049	Spearman: 0.0349
2022-11-30 00:25:09 - Manhattan-Distance:	Pearson: 0.0533	Spearman: 0.0562
2022-11-30 00:25:09 - Euclidean-Distance:	Pearson: -0.0083	Spearman: -0.0045
2022-11-30 00:25:09 - Dot-Product-Similarity:	Pearson: 0.1239	Spearman: 0.1413
2022-11-30 00:25:09 - Save model to static/output/models/microsoft/mpnet-base-2022-11-30_00-24-04-mean-sts


...

2022-11-30 00:35:47 - Cosine-Similarity :	Pearson: 0.9635	Spearman: 0.9203
2022-11-30 00:35:47 - Manhattan-Distance:	Pearson: 0.9351	Spearman: 0.9203
2022-11-30 00:35:47 - Euclidean-Distance:	Pearson: 0.9311	Spearman: 0.9203
2022-11-30 00:35:47 - Dot-Product-Similarity:	Pearson: 0.9592	Spearman: 0.9220


Epoch:  75%|███████▌  | 3/4 [10:50<02:38, 158.89s/it]

2022-11-30 00:35:48 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:


Epoch: 100%|██████████| 4/4 [10:56<00:00, 164.02s/it]

2022-11-30 00:35:53 - Cosine-Similarity :	Pearson: 0.9635	Spearman: 0.9203
2022-11-30 00:35:53 - Manhattan-Distance:	Pearson: 0.9350	Spearman: 0.9203
2022-11-30 00:35:53 - Euclidean-Distance:	Pearson: 0.9311	Spearman: 0.9203
2022-11-30 00:35:53 - Dot-Product-Similarity:	Pearson: 0.9591	Spearman: 0.9220


## Evaluate on a question

In [87]:
# qa_pairs = QA.loc[:, "question":"answer"]

In [36]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel

model_ckpt = "static/output/models/microsoft/mpnet-base-2022-11-30_00-24-04-mean-sts"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

import torch

device = torch.device("cuda")
model.to(device)

def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

qa_pairs = Dataset.from_pandas(QA)

In [37]:
embeddings_dataset = qa_pairs.map(
    lambda x: {"embeddings": get_embeddings(x["question"]).detach().cpu().numpy()[0]}
)

embeddings_dataset.add_faiss_index(column="embeddings")

100%|██████████| 1/1 [00:00<00:00, 131.19it/s]


Dataset({
    features: ['group', 'question', 'answer', 'embeddings'],
    num_rows: 645
})

In [96]:
# tool.save_py_obj("emb_test.pkl", embeddings_dataset)
# emb_test = tool.load_py_obj("emb_test.pkl")
# emb_test

Dataset({
    features: ['group', 'question', 'answer', 'embeddings'],
    num_rows: 645
})

In [38]:
# question = "terapi untuk penyakit bph adalah"
# question = "Dermatitis Atopik dapat diobati dengan"
question = "terapi untuk penyakit impetigo bulosa"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=4
)

In [39]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

for _, row in samples_df.iterrows():
    print(f"Question : {row.question}")
    print(f"Answer : {row.answer}")
    print(f"SCORE: {row.scores}")
    print("=" * 50)
    print()

Question : bagaimana Impetigo Bulosa dapat diobati?
Answer : R/ Sporetic syr fl I S 2 dd 40 mg R/ Bactoderm gr 10 Mediklin gr 10 mfla da inpot I S 2 dd ue
SCORE: 6.04553747177124

Question : apa terapi yang tepat untuk penyakit Impetigo Bulosa?
Answer : R/ Sporetic syr fl I S 2 dd 40 mg R/ Bactoderm gr 10 Mediklin gr 10 mfla da inpot I S 2 dd ue
SCORE: 4.567631244659424

Question : terapi apa yg disarankan untuk penderita Impetigo Bulosa?
Answer : R/ Sporetic syr fl I S 2 dd 40 mg R/ Bactoderm gr 10 Mediklin gr 10 mfla da inpot I S 2 dd ue
SCORE: 3.652449369430542

Question : terapi apa yang diperlukan untuk penyakit Impetigo Bulosa?
Answer : R/ Sporetic syr fl I S 2 dd 40 mg R/ Bactoderm gr 10 Mediklin gr 10 mfla da inpot I S 2 dd ue
SCORE: 2.1214425563812256



## F1 & Accuracy Evaluation

In [33]:
model_ft = "static/output/models/microsoft/mpnet-base-2022-11-30_00-24-04-mean-sts"
loadFtModel = tool.LoadFTModel(model_ft)

In [34]:
embedding_dataset = loadFtModel.map_embeddings(QA)

100%|██████████| 1/1 [00:00<00:00, 331.04it/s]


In [19]:
tool.save_py_obj(f"embedding_dataset.pkl", embedding_dataset)

In [35]:
loadFtModel.get_evaluation(QA_test)

100%|██████████| 129/129 [00:02<00:00, 46.94it/s]

true prediction = 113, false prediction = 16


'F1 = 0.876, Accuracy = 0.876'

In [21]:
tool.save_py_obj("df_for_calculate_f1_acc.pkl", loadFtModel.evaluation_df)

In [27]:
df_for_calculate_f1_acc = tool.load_py_obj("df_for_calculate_f1_acc.pkl")